In [18]:
import pandas as pd
import numpy as np
import json
from selenium import webdriver
import requests
import time
from tqdm import tqdm

# Call RAWG API
    1. Access API
    2. Compile the necessary data
    3. Save the data

In [2]:
# the rawg api call url
# example: requests.get('https://api.rawg.io/api/games?page=1&page_size=40').json()
rawg_api = 'https://api.rawg.io/api/games' # ?page=<number>&page_size=<number>

In [3]:
def get_genres(result):
    '''
    Input:
        results: RAWG video game API 'results' json
    Returns:
        list of game genres
    '''
    return [genre['name'] for genre in result['genres']]

In [4]:
def list_games(json):
    '''
    Input:
        json: RAWG video game API 'results' json
    Returns:
        A dictionary containing the games'  id, name
        rating, ratings breakdown, release date,
        metacritic score, ratings count, and genres
    '''
    games = []
    for result in json['results']:
        games_dict = {}
        genres = get_genres(result)
        
        games_dict['id'] = result['id']
        games_dict['name'] = result['name']
        games_dict['rating'] = result['rating']
        games_dict['ratings'] = result['ratings']
        games_dict['released'] = result['released']
        games_dict['metacritic'] = result['metacritic']
        games_dict['ratings_count'] = result['ratings_count']
        games_dict['genres'] = genres
        
        # if more than 3 people voted for the rating save the data
        if games_dict['ratings_count'] > 3:
            games.append(games_dict)
    return games

In [5]:
def to_json(game, filename):
    '''
    Input:
        game: dictionary containing the data to write to the file
        filename: a string stating the name and path you wish to save the file at
    Returns:
        json file
    '''
    with open(f'{filename}.json', 'a') as file:
            file.write(f'{json.dumps(game)}\n')
    return

In [6]:
def call_api(api, pages, filename):
    '''
   Input:
       api: rawg api url
       pages: which pages I want to scrape from the api
       filename: the filename and path I want to save the file
    Returns:
        json file containing the the games'  id, name
        rating, ratings breakdown, release date,
        metacritic score, ratings count and genres, and a list
        of games that failed to save to json    
    '''
    not_added =[]
    total_games = []
    
    # make and api call extract what you need
    for page in tqdm(range(pages[0], pages[1])):
        json = requests.get(f'{api}?page={page}&page_size=40').json()
        games = list_games(json)
        try: # try save the file to json
            for game in games: 
                to_json(game, filename)
        except: # if failed, save in a python list to address later
            not_added.append(game)
        time.sleep(1)       
    return not_added

In [7]:
not_added = call_api(rawg_api, [1, 8300], 'raw_data')

100%|████████████████████████████████████████████████████████████████████████████| 8299/8299 [4:37:58<00:00,  2.01s/it]


In [7]:
df = pd.read_json('raw_data.json', lines=True)
df.head()

,id,name,rating,ratings,released,metacritic,ratings_count,genres
0,3498,Grand Theft Auto V,4.48,"[{'id': 5, 'title': 'exceptional', 'count': 16...",2013-09-17,96.0,2758,"[Action, Shooter]"
1,4200,Portal 2,4.61,"[{'id': 5, 'title': 'exceptional', 'count': 16...",2011-04-19,95.0,2405,"[Shooter, Puzzle]"
2,3328,The Witcher 3: Wild Hunt,4.68,"[{'id': 5, 'title': 'exceptional', 'count': 20...",2015-05-18,93.0,2552,[RPG]
3,5679,The Elder Scrolls V: Skyrim,4.39,"[{'id': 5, 'title': 'exceptional', 'count': 10...",2011-11-11,NaN,2015,"[Action, RPG]"
4,12020,Left 4 Dead 2,4.08,"[{'id': 4, 'title': 'recommended', 'count': 76...",2009-11-17,89.0,1431,"[Action, Shooter]"


In [8]:
print(f'{df.describe()}\n')
print(f'{df.info()}\n')
print(f"released NA {df['released'].isna().sum()}\n")
print(f"metacritic NA {df['metacritic'].isna().sum()}\n")

                  id        rating   metacritic  ratings_count
count   11298.000000  11298.000000  2271.000000   11298.000000
mean    34967.389715      2.717164    73.616028      54.676226
std     57025.162227      1.504177    10.458881     144.520690
min         2.000000      0.000000    24.000000       4.000000
25%     10665.500000      2.000000    68.000000       6.000000
50%     19600.500000      3.250000    75.000000      13.000000
75%     38375.500000      3.870000    81.000000      36.000000
max    394092.000000      5.000000    97.000000    2758.000000

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11298 entries, 0 to 11297
Data columns (total 8 columns):
id               11298 non-null int64
name             11298 non-null object
rating           11298 non-null float64
ratings          11298 non-null object
released         11088 non-null object
metacritic       2271 non-null float64
ratings_count    11298 non-null int64
genres           11298 non-null object
dtypes: float

In [9]:
# view which games are missing release dates and order it by how many people it was rated
df[df.released.isna()].sort_values(by='ratings_count', ascending=False).head(25)

,id,name,rating,ratings,released,metacritic,ratings_count,genres
226,5687,The Playroom,2.21,"[{'id': 1, 'title': 'skip', 'count': 210, 'per...",None,NaN,441,[]
548,42309,Dead Island: Epidemic,2.20,"[{'id': 1, 'title': 'skip', 'count': 63, 'perc...",None,NaN,136,[Arcade]
794,42391,Nosgoth,2.54,"[{'id': 1, 'title': 'skip', 'count': 40, 'perc...",None,NaN,97,"[Action, Shooter]"
1491,10096,3DMark,3.43,"[{'id': 4, 'title': 'recommended', 'count': 44...",None,NaN,79,[]
1247,23576,GOD EATER RESURRECTION,3.20,"[{'id': 3, 'title': 'meh', 'count': 28, 'perce...",None,NaN,71,[Action]
2516,9551,Deep Rock Galactic,4.14,"[{'id': 4, 'title': 'recommended', 'count': 40...",None,NaN,61,"[Action, Indie]"
2104,10156,Aliens: Colonial Marines Collection,2.60,"[{'id': 1, 'title': 'skip', 'count': 21, 'perc...",None,45.0,54,[Action]
4212,5673,Terminator Salvation,2.70,"[{'id': 3, 'title': 'meh', 'count': 21, 'perce...",None,NaN,40,"[Action, Shooter]"
807,18785,Sam & Max 303: They Stole Max's Brain!,2.76,"[{'id': 1, 'title': 'skip', 'count': 13, 'perc...",None,NaN,38,[Adventure]
3325,11587,Kenshi,4.16,"[{'id': 5, 'title': 'exceptional', 'count': 16...",None,NaN,38,"[Action, RPG, Strategy, Simulation, Indie]"


# Cleaning the dataset
    1. Filling or removing missing values
    2. Save the new dataset to a new file

In [10]:
# check which game is missing release date and manually fill those with over 30 votes
def add_release_dates(df):
    '''
    This function fills specific observations with specific release dates.
    Input:
        df: Original raw data dataframe
    Output:
        Partially filled release dates in the dataframe
    '''
    # star wars: obi wan
    df.at[11169, 'released'] = '2001-12-19'
    # god eater resurrection
    df.at[1247, 'released'] = '2015-10-29' 
    # deep rock galactic
    df.at[2516, 'released'] = '2018-02-28'
    # aliens: Colonial Marines Collection
    df.at[2104, 'released'] = '2013-02-12'
    # terminator Salvation
    df.at[4212, 'released'] = '2009-05-01'
    # kenshi
    df.at[3325, 'released'] = '2018-12-06'
    # Sam & Max 303: They Stole Max's Brain!
    df.at[807, 'released'] = '2010-06-22'
    # Sam & Max 303: Beyond the Alley of the Dolls
    df.at[812, 'released'] = '2010-07-20'
    # Sam & Max 303: The Penal Zone
    df.at[814, 'released'] = '2010-04-02'
    # Sam & Max 303: The City That Dares Not Sleep
    df.at[811, 'released'] = '2010-08-30'
    # Sam & Max 303:  The Tomb of Sammun-Mak
    df.at[813, 'released'] = '2010-05-18'
    # Sword of the Stars: The Pit
    df.at[2396, 'released'] = '2013-02-21'
    # SpellForce 2 Anniversary Edition
    df.at[2399, 'released'] = '2017-04-12'
    # raft
    df.at[4281, 'released'] = '2018-05-23'
    # Unheard
    df.at[5799, 'released'] = '2019-03-29'
    # Hector: Episode 2
    df.at[1167, 'released'] = '2011-06-25'
    
    # most of the games with higher vote count that are missing release date are closed, 
    #suspended or aren't rated.
    df.dropna(subset=['released'], inplace=True)
    return df

# drop games that do not contain genres
df = df[df['genres'].map(lambda d: len(d)) > 0]

# add some release dates
df = add_release_dates(df)

# asses the NAN value situation
print(f'Missing metacritic values: {df['metacritic'].isna().sum()}\n')
print(f'{df.info()}/n')

Missing metacritic values: 8839

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11104 entries, 0 to 11297
Data columns (total 8 columns):
id               11104 non-null int64
name             11104 non-null object
rating           11104 non-null float64
ratings          11104 non-null object
released         11104 non-null object
metacritic       2265 non-null float64
ratings_count    11104 non-null int64
genres           11104 non-null object
dtypes: float64(2), int64(2), object(4)
memory usage: 780.8+ KB
None/n


## Filling the metacritic scores for as many games as possible
    1. Retrieve the names of all games without metacritic score
    2. Scrape the missing values

In [11]:
# list all the game without metacritic scores
no_metacritic = np.array(df['name'][df['metacritic'].isna()])

In [16]:
# scrape the missing metacritic scores using selenium
def scrape_metacritic(names, filename):
    '''
    Input:
        names: list of games to scrape their score from metacritic
        filename: a string for saving the json file
    Returns:
        json file containing a list of dictionarys with the name of the game and the 
    '''
    scores = []
    for name in tqdm(names):
        game = {}
        # open metacritic
        driver = webdriver.Chrome()
        driver.get("https://www.metacritic.com/game")
        driver.implicitly_wait(14)
        
        try:
            # find search bar to search the, enter the game name, and click search
            driver.find_element_by_id('primary_search_box').send_keys(f'{name}')
            driver.implicitly_wait(2)
            driver.find_element_by_id('primary_menu_item_enter_search').click()

            # filter out non games
            driver.implicitly_wait(6)
            parent = driver.find_element_by_class_name("filter_area")
            driver.implicitly_wait(2)
            element = parent.find_elements_by_class_name("title")[2].click()
            
            # good reviews
            if driver.find_elements_by_xpath('(.//span[@class = "metascore_w medium game positive"][1])'):
                driver.implicitly_wait(0.5)
                score = driver.find_elements_by_xpath('(.//span[@class = "metascore_w medium game positive"][1])')[0].text
            
            # no review yet
            elif driver.find_elements_by_xpath('(.//span[@class = "metascore_w medium game tbd"][1])'):
                driver.implicitly_wait(0.5)
                score = driver.find_elements_by_xpath('(.//span[@class = "metascore_w medium game tbd"][1])')[0].text
            
            # mixed reviews
            elif driver.find_elements_by_xpath('(.//span[@class = "metascore_w medium game mixed"][1])'):
                driver.implicitly_wait(0.5)
                score = driver.find_elements_by_xpath('(.//span[@class = "metascore_w medium game mixed"][1])')[0].text
            
            # bad reviews
            elif driver.find_elements_by_xpath('(.//span[@class = "metascore_w medium game negative"][1])'):
                driver.implicitly_wait(0.5)
                score = driver.find_elements_by_xpath('(.//span[@class = "metascore_w medium game negative"][1])')[0].text
            
            else:
                score = "Not Found"             
        except:
            score = "Selenium Fail" 
        
        game[name] = score
        to_json(game, filename)
        scores.append(game)    
        driver.close()
    
    driver.quit()
    return scores

# last scraped 900-1100
metacritic_scores = scrape_metacritic(no_metacritic[900:1100], 'metacritic_scores')

100%|██████████████████████████████████████████████████████████████████████████████| 200/200 [1:15:18<00:00, 22.59s/it]


In [17]:
metacritic_scores

[{'NEKOPARA Vol. 0': 'tbd'},
 {'Call to Arms': 'tbd'},
 {'Hurtworld': 'tbd'},
 {'The Council': '79'},
 {'Interplanetary': '84'},
 {'Marble Mayhem: Fragile Ball': 'Not Found'},
 {'Secret Of Magia': 'tbd'},
 {'DOOM Eternal': 'tbd'},
 {'Pressured': 'tbd'},
 {'Splatter - Zombie Apocalypse': 'Not Found'},
 {"Spooky's Jump Scare Mansion": 'tbd'},
 {'Iron Snout': 'tbd'},
 {'MDK': '83'},
 {'Tooth and Tail': '80'},
 {'Carmageddon: Max Damage': 'tbd'},
 {'Interplanetary: Enhanced Edition': 'tbd'},
 {'Platypus 2': 'Not Found'},
 {'Streets of Rage 2': '84'},
 {'Stargunner': 'tbd'},
 {'Rolling Sun': 'tbd'},
 {'Subterrain': 'tbd'},
 {'Umbrella Corps /Biohazard Umbrella Corps': 'Not Found'},
 {'Paranautical Activity: Deluxe Atonement Edition': 'Not Found'},
 {'Normality': 'tbd'},
 {'Distance': '75'},
 {'Hohokum': '75'},
 {'Breach & Clear': 'tbd'},
 {'Remnant: From the Ashes': '76'},
 {'EA SPORTS UFC 3': '75'},
 {'Business Tour': 'tbd'},
 {'Golf With Your Friends': 'tbd'},
 {'Town of Salem': 'tbd'},
 

In [ ]:
raw_scores_df = pd.read_csv('missing_metacritic.csv')

def extract_selenium_fail(scores):
    return [del scores[k] for k,v in scores.item() if v == 'Selenium Fail'
    
def fill_metacritic(df, scores, column):
    
    # create a single column df with all the scraped data found on the diagonal
    scores_df.set_index([scores.columns], inplace=True)
    scores_df = pd.DataFrame(np.diag(scores_df), index=scores_df.index, columns=['metacritic'])
    
    # fill the missing results from the scraped scores by index
    df.set_index(column, inplace=True)
    df.metacritic = df['metacritic'].fillna(scores_df['metacritic'])
    return df